モデル生成用ノートブック  
LB0.794 PB0.796位だったと思います

In [ ]:
%load_ext Cython

In [ ]:
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
import lightgbm as lgb
import pickle
import datetime
import collections
from sklearn.preprocessing import LabelEncoder
import random
import os
import pickle

In [ ]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

seed_everything(707)

In [ ]:
train_pickle = '../input/riiid-cross-validation-files/cv1_train.pickle'
valid_pickle = '../input/riiid-cross-validation-files/cv1_valid.pickle'
question_file = '../input/riiid-test-answer-prediction/questions.csv'
debug = False
build = True
root = '../input/base-0794-20201224output/'

# read data
feld_needed = ['row_id','timestamp', 'user_id', 'content_id', 'content_type_id', 'answered_correctly', 'prior_question_elapsed_time', 'prior_question_had_explanation','user_answer']

In [ ]:
# Transform tags into lists of ints:
questions_df = pd.read_csv(question_file)
questions_df['part'] = (questions_df['part'] - 1).astype('uint8')
questions_df['correct_answer'] = questions_df['correct_answer'].astype('uint8')
questions_df['tags'] = questions_df['tags'].apply(lambda ts: [int(x) for x in str(ts).split() if x != 'nan'])

tag_rank = []
tag_columns = []
tag_to_questions = {}
for i, row in questions_df.iterrows():
    for t in row['tags']:
        tag_rank.append(t)
        if t not in tag_to_questions:
            tag_to_questions[t] = set()
        tag_to_questions[t].add(row['question_id'])
tags_df = pd.DataFrame([{'tag':t,'questions':qs}for t,qs in tag_to_questions.items()])
tag_rank, counts = zip(*collections.Counter(tag_rank).most_common(1))
print(tag_rank)
for t in tag_rank:
    tag_columns.append('tags_' + str(t))
    for i in range(len(questions_df)):
        if t in questions_df.iloc[i]['tags']:
            questions_df.at[i,'tags_' + str(t)] = 1
        else:
            questions_df.at[i,'tags_' + str(t)] = 0
    questions_df['tags_' + str(t)] = questions_df['tags_' + str(t)].astype('uint8')

del questions_df['bundle_id']
print(tag_columns)

le = LabelEncoder()
encoded = le.fit_transform(questions_df['tags'].astype(str))
decoded = le.inverse_transform(encoded)
questions_df['enc_tags'] = encoded.astype('uint16')
del questions_df['tags'], le
questions_df = questions_df.set_index('question_id')
questions_df.index.name = 'content_id'

questions_df.head(3)

In [ ]:
train = pd.read_pickle(train_pickle)[feld_needed]
valid = pd.read_pickle(valid_pickle)[feld_needed]
tmp = pd.concat([train[['content_type_id','content_id','answered_correctly']],valid[['content_type_id','content_id','answered_correctly']]])
content_df = tmp.loc[tmp['content_type_id']==0][['content_id','answered_correctly']].groupby(['content_id']).agg(['mean']).reset_index()
content_df.columns = ['content_id', 'answered_correctly_avg_c']
content_df['answered_correctly_avg_c'] = (content_df['answered_correctly_avg_c'] * 100).astype(np.uint8)
content_df = content_df.set_index('content_id')
content_df.index.name = 'content_id'
content_df.to_csv('content_df.csv')
del content_df,tmp,train,valid
_=gc.collect()

In [ ]:
%%cython
import cython
cimport cython
import numpy as np
cimport numpy as np
import pandas as pd
import gc

DTYPE = np.int32
ctypedef np.int32_t np_int_t
#dict
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
def add_user_feats(df, 
                   answer_per_dict,
                   answered_correctly_sum_u_dict,
                   answered_correctly_cumsum_u_dict,
                   answered_incorrectly_cumsum_u_dict,
                   count_u_dict,
                   parts_u_dict,
                   parts_count_u_dict,
                   answered_diff_sum_u_dict,
                   avg_c_sum_u_dict,
                   parts_avg_c_dict,
                   user_answer_per_sum_dict,
                   parts_user_answer_per_sum_dict,
                   content_correct_user_mean_dict,
                   content_correct_count_dict,
                   parts_content_correct_user_mean_dict,
                   last_correct_timestamp_dict,
                   last_incorrect_timestamp_dict,
                   like_answer_dict,
                   like_answer_three_dict,
                   dislike_answer_dict,
                   dislike_answer_three_dict,
                   parts_answered_correctly_cumsum_u_dict,
                   parts_answered_incorrectly_cumsum_u_dict):
    
    cdef int arr_size = len(df)
    
    cdef np.ndarray[np_int_t, ndim=1] acsu = np.zeros(arr_size, dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] accu = np.zeros(arr_size, dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] aicu = np.zeros(arr_size, dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] cu = np.zeros(arr_size, dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=2] ptu = np.zeros([arr_size,7], dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=2] ptcu = np.zeros([arr_size,7], dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] paccu = np.zeros(arr_size, dtype=DTYPE)#
    cdef np.ndarray[np_int_t, ndim=1] paicu = np.zeros(arr_size, dtype=DTYPE)#
    cdef np.ndarray[np_int_t, ndim=1] adsu = np.zeros(arr_size, dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] avcu = np.zeros(arr_size, dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] pavc = np.zeros(arr_size, dtype=DTYPE)
    cdef np.ndarray[float, ndim=1] uaps = np.zeros(arr_size,dtype=np.float32)
    cdef np.ndarray[float, ndim=1] puaps = np.zeros(arr_size,dtype=np.float32)
    cdef np.ndarray[float, ndim=1] cucm = np.zeros(arr_size,dtype=np.float32)
    cdef np.ndarray[float, ndim=1] pcucm = np.zeros(arr_size,dtype=np.float32)
    cdef np.ndarray[long, ndim=1] lct = np.zeros(arr_size, dtype=long)
    cdef np.ndarray[long, ndim=1] lit = np.zeros(arr_size, dtype=long)
    cdef np.ndarray[float, ndim=1] like = np.zeros(arr_size, dtype=np.float32)
    cdef np.ndarray[float, ndim=1] dislike = np.zeros(arr_size, dtype=np.float32)
    cdef int cnt, i, j
    cdef np.ndarray[long, ndim=1] row
    #                                0             1              2                3                     4           5            6                7                     
    for cnt,row in enumerate(df[['user_id','answered_correctly','part','answered_correctly_avg_c','content_id','user_answer','timestamp','correct_answer']].values):
        acsu[cnt] = answered_correctly_sum_u_dict[row[0]]
        accu[cnt] = answered_correctly_cumsum_u_dict[row[0]]
        aicu[cnt] = answered_incorrectly_cumsum_u_dict[row[0]]
        paccu[cnt] = parts_answered_correctly_cumsum_u_dict[row[2]][row[0]]
        paicu[cnt] = parts_answered_incorrectly_cumsum_u_dict[row[2]][row[0]]
        cu[cnt] = count_u_dict[row[0]]
        adsu[cnt] = answered_diff_sum_u_dict[row[0]]
        avcu[cnt] = avg_c_sum_u_dict[row[0]]
        uaps[cnt] = user_answer_per_sum_dict[row[0]]
        lct[cnt] = row[6] - last_correct_timestamp_dict[row[0]]
        lit[cnt] = row[6] - last_incorrect_timestamp_dict[row[0]]

        if row[2] == 1:
            if (row[6] == 0) or (dict_sub(answered_correctly_sum_u_dict[row[0]] ,count_u_dict[row[0]])==0):
                like[cnt] = np.nan
                dislike[cnt] = np.nan
            else:
                like[cnt] = like_answer_three_dict[row[7]][row[0]] / dict_sub(cu[cnt] ,acsu[cnt])
                dislike[cnt] = dislike_answer_three_dict[row[7]][row[0]] / dict_sub(cu[cnt] ,acsu[cnt])
        else:
            if (row[6] == 0) or (dict_sub(answered_correctly_sum_u_dict[row[0]] , count_u_dict[row[0]])==0):
                like[cnt] = np.nan
                dislike[cnt] = np.nan
            else:
                like[cnt] = like_answer_dict[row[7]][row[0]] / dict_sub(cu[cnt] , acsu[cnt])
                dislike[cnt] = dislike_answer_dict[row[7]][row[0]] / dict_sub(cu[cnt] , acsu[cnt])
        if content_correct_count_dict[row[4]] > 0:
            cucm[cnt] = content_correct_user_mean_dict[row[4]] / content_correct_count_dict[row[4]]
            pcucm[cnt] = parts_content_correct_user_mean_dict[row[4]] / content_correct_count_dict[row[4]]
        for i in range(7):
            ptu[cnt,i] = parts_u_dict[i][row[0]]
            ptcu[cnt,i] = parts_count_u_dict[i][row[0]]
            if i == row[2]:
                pavc[cnt] = parts_avg_c_dict[i][row[0]] / dict_sum(parts_count_u_dict[i][row[0]] , 1)
                parts_avg_c_dict[i][row[0]] = dict_sum(parts_avg_c_dict[row[2]][row[0]],row[3])
                
        if ptcu[cnt,row[2]] != 0:
            puaps[cnt] = parts_user_answer_per_sum_dict[row[2]][row[0]] / ptcu[cnt,row[2]]
        
        if row[1] == 1:
            answered_correctly_sum_u_dict[row[0]] = dict_sum(answered_correctly_sum_u_dict[row[0]],1)
            answered_correctly_cumsum_u_dict[row[0]] = dict_sum(answered_correctly_cumsum_u_dict[row[0]],100-row[3])
            answered_incorrectly_cumsum_u_dict[row[0]] = 0
            last_correct_timestamp_dict[row[0]] = row[6]
            parts_answered_correctly_cumsum_u_dict[row[2]][row[0]] = dict_sum(parts_answered_correctly_cumsum_u_dict[row[2]][row[0]],100-row[3])
            parts_answered_incorrectly_cumsum_u_dict[row[2]][row[0]] = 0
        else:
            answered_correctly_cumsum_u_dict[row[0]] = 0
            answered_incorrectly_cumsum_u_dict[row[0]] = dict_sum(answered_incorrectly_cumsum_u_dict[row[0]],row[3])
            last_incorrect_timestamp_dict[row[0]] = row[6]
            parts_answered_correctly_cumsum_u_dict[row[2]][row[0]] = 0
            parts_answered_incorrectly_cumsum_u_dict[row[2]][row[0]] = dict_sum(parts_answered_incorrectly_cumsum_u_dict[row[2]][row[0]],row[3]) 
            if row[2] == 1:
                #answer0,1,3
                like_answer_three_dict[row[5]][row[0]] = dict_sum(like_answer_three_dict[row[5]][row[0]],1)
                for j in [0,1,3]:
                    if row[5] != j:
                        dislike_answer_three_dict[j][row[0]] = dict_sum(dislike_answer_three_dict[j][row[0]],1)
            else:
                like_answer_dict[row[5]][row[0]] = dict_sum(like_answer_dict[row[5]][row[0]],1)
                for j in [0,1,2,3]:
                    if row[5] != j:
                        dislike_answer_dict[j][row[0]] = dict_sum(dislike_answer_dict[j][row[0]],1)
            
            
            
        answered_diff_sum_u_dict[row[0]] = dict_sum(answered_diff_sum_u_dict[row[0]],abs(row[3] - (row[1] * 100)))
        count_u_dict[row[0]] = dict_sum(count_u_dict[row[0]],1)
        avg_c_sum_u_dict[row[0]] = dict_sum(avg_c_sum_u_dict[row[0]],row[3])
        
        if row[4] in answer_per_dict[row[5]]:
            user_answer_per_sum_dict[row[0]] = user_answer_per_sum_dict[row[0]] + answer_per_dict[row[5]][row[4]]
            parts_user_answer_per_sum_dict[row[2]][row[0]] = parts_user_answer_per_sum_dict[row[2]][row[0]] + answer_per_dict[row[5]][row[4]]
        else:
            user_answer_per_sum_dict[row[0]] = user_answer_per_sum_dict[row[0]] + 0.33
            parts_user_answer_per_sum_dict[row[2]][row[0]] = parts_user_answer_per_sum_dict[row[2]][row[0]] + 0.33
       
        parts_u_dict[row[2]][row[0]] = dict_sum(parts_u_dict[row[2]][row[0]],row[1])
        parts_count_u_dict[row[2]][row[0]] = dict_sum(parts_count_u_dict[row[2]][row[0]],1)
        
        if row[1] == 1:
            content_correct_count_dict[row[4]] =  dict_sum(content_correct_count_dict[row[4]],1)
            content_correct_user_mean_dict[row[4]] = content_correct_user_mean_dict[row[4]] + (answered_correctly_sum_u_dict[row[0]] / count_u_dict[row[0]])
            parts_content_correct_user_mean_dict[row[4]] = parts_content_correct_user_mean_dict[row[4]] + (parts_u_dict[row[2]][row[0]] / parts_count_u_dict[row[2]][row[0]])
 
    df['answered_correctly_sum_u'] = acsu
    df['answered_correctly_sum_u'] = df['answered_correctly_sum_u'].astype('uint16')
    df['answered_cumsum_u'] = accu - aicu
    df['answered_cumsum_u'] = df['answered_cumsum_u'].astype('int16')
    df['part_answered_cumsum_u'] = paccu - paicu
    df['part_answered_cumsum_u'] = df['part_answered_cumsum_u'].astype('int16')    
    df['count_u'] = cu
    df['count_u'] = df['count_u'].astype('uint16')
    df['answered_correctly_avg_u'] = df['answered_correctly_sum_u'] / df['count_u']
    df['answered_correctly_avg_u'] = df['answered_correctly_avg_u'].astype('float16')
    df['answered_diff_mean'] = adsu  / cu
    df['answered_diff_mean'] = df['answered_diff_mean'].astype('float16')
    df['avg_c_mean'] = avcu / cu
    df['avg_c_mean'] = df['avg_c_mean'].astype('float16')
    df['part_avg_c_mean'] = pavc
    df['part_avg_c_mean'] = df['part_avg_c_mean'].astype('uint8')
    
    df['avg_c_per_u'] = df['avg_c_mean'] / (df['answered_correctly_avg_u'] * 100)
    df['avg_c_per_u'] = df['avg_c_per_u'].astype('float16')
    
    df['user_answer_per_mean'] = uaps
    df['user_answer_per_mean'] = df['user_answer_per_mean']  / df['count_u']
    df['user_answer_per_mean'] = df['user_answer_per_mean'].astype('float16')
    
    df['part_user_answer_per_mean']= puaps
    df['part_user_answer_per_mean'] = df['part_user_answer_per_mean'].astype('float16')
    
    df['content_lv'] = cucm
    df['content_lv'] = df['content_lv'].astype('float16')
    df['part_content_lv'] = pcucm
    df['part_content_lv'] = df['part_content_lv'].astype('float16')
    df.loc[df['content_lv']==0,'content_lv']=0.5
    df.loc[df['part_content_lv']==0,'part_content_lv']=0.5

    df['last_correct_timelag'] = lct
    df['last_correct_timelag'] = df['last_correct_timelag'].astype('uint32')
    df['last_incorrect_timelag'] = lit
    df['last_incorrect_timelag'] = df['last_incorrect_timelag'].astype('uint32')
    
    df['is_like_answer'] = like
    df['is_like_answer'] = df['is_like_answer'].astype('float16')
    df['is_dislike_answer'] = dislike
    df['is_dislike_answer'] = df['is_dislike_answer'].astype('float16')
    df['part_count_per'] = 0
    df['lr_count_per'] = 0
    cdef str pnum
    for i in range(7):
        pnum = str(i)
        df['p' + pnum + '_count_u'] = ptcu[:,i]
        df['p' + pnum + '_count_u'] = df['p' + pnum + '_count_u']
        df['p' + pnum + '_count_u'] = df['p' + pnum + '_count_u'].astype('uint32')
        df['p' + pnum + '_mean_u'] = ptu[:,i] / ptcu[:,i]
        df['p' + pnum + '_mean_u']  = df['p' + pnum + '_mean_u'] * (df['p' + pnum + '_count_u'] / df['count_u'])
        df['p' + pnum + '_mean_u'] = df['p' + pnum + '_mean_u'].astype('float16')
        df.loc[df['part']==i,'part_count_per'] = df['p' + pnum + '_count_u'] / df['count_u']
    df['part_count_per'] = df['part_count_per'].astype('float16')    
    df.loc[df['part']<4,'lr_count_per'] = ((df['p0_count_u'] + df['p1_count_u'] + df['p2_count_u'] + df['p3_count_u']) / df['count_u'])
    df.loc[df['part']>3,'lr_count_per'] = ((df['p4_count_u'] + df['p5_count_u'] + df['p6_count_u']) / df['count_u']).astype('float16')
    df['lr_count_per'] = df['lr_count_per'].astype('float16')
    df.replace([np.inf, -np.inf], np.nan,inplace=True)
    return df

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef int dict_sum(int a, int b):
    return a + b
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef int dict_sub(int a, int b):
    return a - b

In [ ]:
%%cython
import cython
cimport cython
import numpy as np
cimport numpy as np
import pandas as pd

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
def add_time_feats(df,time_u_dict,lect_u_dict):
    cdef int arr_size = len(df)
    cdef int cnt
    cdef np.ndarray[long, ndim=1] row
    cdef np.ndarray[long, ndim=1] tu = np.zeros(arr_size,dtype=long)
    cdef np.ndarray[long, ndim=1] lc = np.zeros(arr_size,dtype=long)
    for cnt,row in enumerate(df[['user_id','timestamp','content_type_id']].values):
        if (row[1] - time_u_dict[row[0]]>0):
            tu[cnt] = dict_sub(row[1],time_u_dict[row[0]])
        elif (row[1] == 0):
            tu[cnt] = 0
        else:
            tu[cnt] = tu[cnt - 1]
        lc[cnt] = lect_u_dict[row[0]]
        
        time_u_dict[row[0]] = row[1]
        if (row[2] == 1):
            lect_u_dict[row[0]] = lect_u_dict[row[0]] + 1
    
    cdef int split = 60*60*24
    cdef np.ndarray[long, ndim=1] tu_day = tu // split
    cdef np.ndarray[long, ndim=1] tu_time = tu % split 

    df['lag_time'] = tu_time
    df['lag_time'] = df['lag_time'].astype('uint16')
    df['lag_day'] = tu_day
    df['lag_day'] = df['lag_day'].astype('uint16')
    df.loc[df['lag_day']>0,'lag_time'] = np.iinfo(np.uint16).max
    df['lecture_count'] = lc
    df.loc[df['lecture_count']>np.iinfo(np.uint8).max,'lecture_count'] = np.iinfo(np.uint8).max
    df['lecture_count'] = df['lecture_count'].astype('uint8')
    return df
            
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef int dict_sub(long a, int b):
    return a - b

In [ ]:
#content_answer_per生成
train = pd.read_pickle(train_pickle)[feld_needed]
train = train.loc[train['content_type_id']==0][['content_id','user_answer']]
tmp = train.groupby('content_id').count()
tmp.rename(columns={'user_answer':'count'},inplace=True)
train = pd.read_pickle(train_pickle)[feld_needed]
train = train.loc[train['content_type_id']==0][['content_id','user_answer','content_type_id']]
tmp2 = train.groupby(['content_id','user_answer']).count().reset_index()
tmp2 = tmp2.merge(tmp,left_on='content_id',right_index=True,how='left')
tmp2['answer_per'] = tmp2['content_type_id'] / tmp2['count']
tmp2 = tmp2[['content_id','user_answer','answer_per']]
tmp2['answer_per'].fillna(0.3,inplace=True)
answer_per_dict = {}
for i in range(4):
    answer_per_dict[i] = tmp2.loc[tmp2['user_answer']==i].set_index('content_id')[['answer_per']].to_dict()['answer_per']
del train, tmp2, tmp

In [ ]:
train = pd.read_pickle(train_pickle)[feld_needed]
valid = pd.read_pickle(valid_pickle)[feld_needed]
# answered correctly average for each content
# content_type_idが異なっていて同じコンテンツIDが存在する
content_df = train.loc[train['content_type_id']==0][['content_id','answered_correctly']].groupby(['content_id']).agg(['mean']).reset_index()
content_df.columns = ['content_id', 'answered_correctly_avg_c']
content_df['answered_correctly_avg_c'] = (content_df['answered_correctly_avg_c'] * 100).astype(np.uint8)
content_df = content_df.set_index('content_id')
content_df.index.name = 'content_id'

if debug:
    train = train[:1000000]
    valid = valid[:10000]
else:
    #user_id split because user trace
    #current active user trace 
    print('all =',train['row_id'].min(),train['row_id'].max())
    train = train.sort_values('row_id')
    train = train[int(len(train)/2):]
    print('current =',train['row_id'].min(),train['row_id'].max())
    #usersグループを３つ作成し、それぞれのグループ毎にモデル生成
    users = np.random.choice(train['user_id'].unique(), int(len(train['user_id'].unique()) * 8 / 10), replace=True)

    train = pd.read_pickle(train_pickle)[feld_needed]
    train = train.loc[train['user_id'].isin(users)]
    #推論時のアンサンブルモデルの重みに使用
    with open('train_users.pickle', 'wb') as f:
        pickle.dump(users, f)  
print(train.shape)

In [ ]:
%%cython
import cython
cimport cython
import numpy as np
cimport numpy as np
import pandas as pd

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
def data_format(df,questions_df,content_df,float prior_question_elapsed_time_mean):
    df['row_id'] = df['row_id'].astype('uint32')
    df['user_id'] = df['user_id'].astype('int32')
    df['content_type_id'] = df['content_type_id'].astype('uint8')
    df.loc[df['content_type_id'] != 0,'content_id'] = 532 #暫定
    df['content_id'] = df['content_id'].astype('uint16')
    # changing dtype to avoid lightgbm error
    df['prior_question_had_explanation'] = df.prior_question_had_explanation.fillna(False).astype('uint8')
    df['prior_question_elapsed_time'] = df.prior_question_elapsed_time.fillna(prior_question_elapsed_time_mean)
    df['prior_question_elapsed_time'] = (df['prior_question_elapsed_time'] / 1000).astype('uint16')
    df['timestamp'] = (df['timestamp'] / 1000).astype(np.uint32)
    # merge
    df = pd.concat([df.reset_index(drop=True), questions_df.reindex(df['content_id'].values).reset_index(drop=True)], axis=1)
    df = pd.concat([df.reset_index(drop=True), content_df.reindex(df['content_id'].values).reset_index(drop=True)], axis=1)
    return df

In [ ]:
# fill with mean value for prior_question_elapsed_time
# note that `train.prior_question_elapsed_time.mean()` dose not work!
# please refer https://www.kaggle.com/its7171/can-we-trust-pandas-mean for detail.
prior_question_elapsed_time_mean = train.prior_question_elapsed_time.dropna().values.mean()

train = data_format(train,questions_df,content_df,prior_question_elapsed_time_mean)
valid = data_format(valid,questions_df,content_df,prior_question_elapsed_time_mean)

# memory compaction
train.loc[train['answered_correctly'] < 0,'answered_correctly'] = 0
train['answered_correctly'] = train['answered_correctly'].astype('uint8')
train['user_answer'] = train['user_answer'].astype('uint8')
valid.loc[valid['answered_correctly'] < 0,'answered_correctly'] = 0
valid['answered_correctly'] = valid['answered_correctly'].astype('uint8')
valid['user_answer'] = valid['user_answer'].astype('uint8')

In [ ]:
#経過時間(講義列も考慮)
#train add_time_feats = 0:00:43.003496
#valid add_time_feats = 0:00:03.887652
time_u_dict = defaultdict(int)
lect_u_dict = defaultdict(int)

start = datetime.datetime.now()
train = add_time_feats(train,time_u_dict,lect_u_dict)
print('train add_time_feats =',(datetime.datetime.now()- start))
start = datetime.datetime.now()
valid = add_time_feats(valid,time_u_dict,lect_u_dict)
print('valid add_time_feats =',(datetime.datetime.now()- start))

In [ ]:
train = train.loc[train.content_type_id == False].reset_index(drop=True)
valid = valid.loc[valid.content_type_id == False].reset_index(drop=True)

In [ ]:
%%cython
import cython
cimport cython
import numpy as np
cimport numpy as np
import pandas as pd

DTYPE = np.int32
ctypedef np.int32_t np_int_t

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
def add_prior_feats(df, q_stats_dict,q_enc_tag_dict,q_prior_root_dict,
                    prior_content_dict, prior_prior_content_dict, prior_time_dict, 
                    prior_time_per_sum_dict,
                    prior_lag_dict, prior_prior_lag_dict,
                    lag_sum_dict, prior_avg_c_dict, prior_prior_avg_c_dict,
                    part_lag_sum_dict, prior_part_dict):
    cdef int arr_size = len(df)
    cdef int cnt
    cdef np.ndarray[int, ndim=1] row
    cdef np.ndarray[np_int_t, ndim=1] pc = np.zeros(arr_size,dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] ppc = np.zeros(arr_size,dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] eqtag = np.zeros(arr_size,dtype=DTYPE)
    cdef np.ndarray[float, ndim=1] tp = np.zeros(arr_size,dtype=np.float32)
    cdef np.ndarray[np_int_t, ndim=1] pt = np.zeros(arr_size,dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] ul = np.zeros(arr_size,dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] ull = np.zeros(arr_size,dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] pe = np.zeros(arr_size,dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] ls = np.zeros(arr_size,dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] pac = np.zeros(arr_size,dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] ppac = np.zeros(arr_size,dtype=DTYPE)
    cdef np.ndarray[float, ndim=1] ptps = np.zeros(arr_size,dtype=np.float32)
    cdef np.ndarray[np_int_t, ndim=1] pls = np.zeros(arr_size,dtype=DTYPE)
    cdef np.ndarray[float, ndim=1] pr = np.zeros(arr_size,dtype=np.float32)
    
    for cnt,row in enumerate(df[['user_id','content_id','prior_question_elapsed_time','lag_time' ,'prior_question_had_explanation','answered_correctly_avg_c','part']].values):
        ppc[cnt] = prior_prior_content_dict[row[0]]
        pc[cnt] = prior_content_dict[row[0]]
        if prior_prior_content_dict[row[0]] in q_stats_dict:
            pt[cnt] = prior_time_dict[row[0]] / q_stats_dict[prior_prior_content_dict[row[0]]]
        else:
            pt[cnt] = 1
        ls[cnt] = lag_sum_dict[row[0]]
        pac[cnt] = prior_avg_c_dict[row[0]]
        ppac[cnt] = prior_prior_avg_c_dict[row[0]]
        pls[cnt] = part_lag_sum_dict[row[6]][row[0]]
        if (q_enc_tag_dict[pc[cnt]] == q_enc_tag_dict[row[1]]):
            eqtag[cnt] = 1
        else:
            eqtag[cnt] = 0
        if prior_content_dict[row[0]] > 0:
            if (row[1],prior_content_dict[row[0]]) in q_prior_root_dict:
                pr[cnt] = q_prior_root_dict[(row[1],prior_content_dict[row[0]])]
        else:
            if (row[1],-999) in q_prior_root_dict:
                pr[cnt] = q_prior_root_dict[(row[1],-999)]
        if (prior_content_dict[row[0]] > 0) & (prior_content_dict[row[0]] in q_stats_dict):
            tp[cnt] = row[2] / q_stats_dict[prior_content_dict[row[0]]]
            prior_time_per_sum_dict[row[0]] = prior_time_per_sum_dict[row[0]] + tp[cnt]
        else:
            tp[cnt] = 1
        ptps[cnt] = prior_time_per_sum_dict[row[0]]
        
        if prior_content_dict[row[0]] > 0:
            prior_part_dict[row[0]] = row[6]
            part_lag_sum_dict[prior_part_dict[row[0]]][row[0]] = part_lag_sum_dict[prior_part_dict[row[0]]][row[0]] + tp[cnt]
        prior_prior_content_dict[row[0]] = prior_content_dict[row[0]]
        prior_content_dict[row[0]] = row[1]
        prior_time_dict[row[0]] = row[2] #1つ前のコンテンツの回答時間
        ul[cnt] = prior_lag_dict[row[0]]
        ull[cnt] = prior_prior_lag_dict[row[0]]
        prior_prior_lag_dict[row[0]] = prior_lag_dict[row[0]]
        prior_lag_dict[row[0]] = row[3]
        lag_sum_dict[row[0]] = lag_sum_dict[row[0]] + row[3]
        prior_prior_avg_c_dict[row[0]] = prior_avg_c_dict[row[0]]
        prior_avg_c_dict[row[0]] = row[5]
            
    df['prior_content_id'] = pc
    df['prior_content_id'] = df['prior_content_id'].astype('uint16')
    df['prior_content_diff'] = df['content_id'] - df['prior_content_id']
    df.loc[df['timestamp']==0, 'prior_content_diff'] = -999
    #現在のコンテンツ回答前に実施したコンテンツの割合
    df['prior_root'] = pr
    df['prior_root'] = df['prior_root'].astype('float16')
    
    df['is_same_prior'] = eqtag
    df.loc[df['prior_content_id'] == df['content_id'],'is_same_prior'] = df['is_same_prior'] + 2
    df['is_same_prior'] = df['is_same_prior'].astype('uint8')
    df['lag_time_per'] = df['lag_time'] / df['prior_question_elapsed_time'].astype('float32')
    df['elapsed_lag_per'] = tp
    df['elapsed_lag_per'] = df['elapsed_lag_per'].astype('float16')
    df['elapsed_time_per_mean'] = ptps
    df['elapsed_time_per_mean'] = df['elapsed_time_per_mean'].astype('float16')
    
    df['part_elapsed_time_per_mean'] = pls
    df['part_elapsed_time_per_mean'] = df['part_elapsed_time_per_mean'].astype('float16')

    df['prior_prior_lag_time'] = ull
    df['prior_prior_lag_time'] = df['prior_prior_lag_time'].astype('uint16')
    df['prior_lag_time'] = ul
    df['prior_lag_time'] = df['prior_lag_time'].astype('uint16')
    df['lag_lag_time'] = df['lag_time'] / df['prior_lag_time']
    df['lag_lag_time'] = df['lag_lag_time'].astype('float16')
    df['lag_sum'] = ls
    df['prior_avg_c'] = pac
    df['prior_avg_c'] = df['prior_avg_c'].astype('uint8')
    df['prior_prior_avg_c'] = ppac
    df['prior_prior_avg_c'] = df['prior_prior_avg_c'].astype('uint8')
    return df

In [ ]:
#q_stats_dict = pd.read_csv('../input/riiiddataset/question_stats.csv').set_index('content_id')[['q_elapsed_time_mean']].to_dict()['q_elapsed_time_mean']
#correct answer only
q_stats_dict = pd.read_csv('../input/riiiddataset/correct_q_elapsed_time_mean.csv').set_index('content_id')[['correct_q_elapsed_time_mean']].to_dict()['correct_q_elapsed_time_mean']
q_enc_tag_dict = questions_df[['enc_tags']].to_dict()['enc_tags']
q_ans_dict = questions_df[['correct_answer']].to_dict()['correct_answer']
#現在実行中コンテンツ前に実行されたコンテンツの確率
q_prior_root_dict = pd.read_csv('../input/riiiddataset/prior_position_per.csv').fillna(-999).set_index(['content_id','prior_content_id']).to_dict()['prior_position_per']

In [ ]:
#train add_prior_feats = 0:02:22.203537
#valid add_prior_feats = 0:00:12.576596

prior_content_dict = defaultdict(int)
prior_prior_content_dict = defaultdict(int)
prior_time_dict = defaultdict(int)
prior_time_per_sum_dict = defaultdict(int)
prior_lag_dict = defaultdict(int)
prior_prior_lag_dict = defaultdict(int)
lag_sum_dict = defaultdict(int)
prior_avg_c_dict = defaultdict(int)
prior_prior_avg_c_dict = defaultdict(int)
part_lag_sum_dict = {}
for p in range(0,7):
    part_lag_sum_dict[p] = defaultdict(int)

prior_part_dict = defaultdict(int)
start = datetime.datetime.now()
train = add_prior_feats(train,
                        q_stats_dict,
                        q_enc_tag_dict,
                        q_prior_root_dict,
                        prior_content_dict,
                        prior_prior_content_dict,
                        prior_time_dict,
                        prior_time_per_sum_dict,
                        prior_lag_dict,
                        prior_prior_lag_dict,
                        lag_sum_dict,
                        prior_avg_c_dict,
                        prior_prior_avg_c_dict,
                        part_lag_sum_dict,
                        prior_part_dict)
print('train add_prior_feats =',(datetime.datetime.now()- start))
start = datetime.datetime.now()
valid = add_prior_feats(valid,
                        q_stats_dict,
                        q_enc_tag_dict,
                        q_prior_root_dict,
                        prior_content_dict,
                        prior_prior_content_dict,
                        prior_time_dict,
                        prior_time_per_sum_dict,
                        prior_lag_dict,
                        prior_prior_lag_dict,
                        lag_sum_dict,
                        prior_avg_c_dict,
                        prior_prior_avg_c_dict,
                        part_lag_sum_dict,
                        prior_part_dict)
print('valid add_prior_feats =',(datetime.datetime.now()- start))

In [ ]:
del q_prior_root_dict
_=gc.collect()

In [ ]:
#train add_user_feats = 0:07:19.055945
#valid add_user_feats = 0:00:38.820698
#正解数
answered_correctly_sum_u_dict = defaultdict(int)
#連続正答数
answered_correctly_cumsum_u_dict = defaultdict(int)
#連続不正解
answered_incorrectly_cumsum_u_dict = defaultdict(int)
#回答数
count_u_dict = defaultdict(int)
#パート回答数
parts_count_u_dict = {}
parts_u_dict = {}
parts_avg_c_dict = {}
parts_user_answer_per_sum_dict = {}
parts_answered_correctly_cumsum_u_dict = {}
parts_answered_incorrectly_cumsum_u_dict = {}
for p in range(0,7):
    parts_u_dict[p] = defaultdict(int)
    parts_count_u_dict[p] = defaultdict(int)
    parts_avg_c_dict[p] = defaultdict(int)
    parts_user_answer_per_sum_dict[p] = defaultdict(int)
    parts_answered_correctly_cumsum_u_dict[p] = defaultdict(int)
    parts_answered_incorrectly_cumsum_u_dict[p] = defaultdict(int)
    
#回答期待値との差
answered_diff_sum_u_dict = defaultdict(int)
avg_c_sum_u_dict = defaultdict(int)
user_answer_per_sum_dict = defaultdict(int)

content_correct_user_mean_dict = defaultdict(int)
content_correct_count_dict = defaultdict(int)
parts_content_correct_user_mean_dict = defaultdict(int)

last_correct_timestamp_dict = defaultdict(int)
last_incorrect_timestamp_dict = defaultdict(int)

like_answer_dict = {}
like_answer_three_dict = {}
dislike_answer_dict = {}
dislike_answer_three_dict = {}
for p in range(0,4):
    like_answer_dict[p] = defaultdict(int)
    like_answer_three_dict[p] = defaultdict(int)
    dislike_answer_dict[p] = defaultdict(int)
    dislike_answer_three_dict[p] = defaultdict(int)

start = datetime.datetime.now()
train = add_user_feats(train,
                       answer_per_dict,
                       answered_correctly_sum_u_dict, 
                       answered_correctly_cumsum_u_dict,
                       answered_incorrectly_cumsum_u_dict,
                       count_u_dict,
                       parts_u_dict,
                       parts_count_u_dict,
                       answered_diff_sum_u_dict,
                       avg_c_sum_u_dict,
                       parts_avg_c_dict,
                       user_answer_per_sum_dict,
                       parts_user_answer_per_sum_dict,
                       content_correct_user_mean_dict,
                       content_correct_count_dict,
                       parts_content_correct_user_mean_dict,
                       last_correct_timestamp_dict,
                       last_incorrect_timestamp_dict,
                       like_answer_dict,
                       like_answer_three_dict,
                       dislike_answer_dict,
                       dislike_answer_three_dict,
                       parts_answered_correctly_cumsum_u_dict,
                       parts_answered_incorrectly_cumsum_u_dict)

print('train add_user_feats =',(datetime.datetime.now() - start))

start = datetime.datetime.now()
valid = add_user_feats(valid,
                       answer_per_dict,
                       answered_correctly_sum_u_dict,
                       answered_correctly_cumsum_u_dict,
                       answered_incorrectly_cumsum_u_dict,
                       count_u_dict,
                       parts_u_dict,
                       parts_count_u_dict,
                       answered_diff_sum_u_dict,
                       avg_c_sum_u_dict,
                       parts_avg_c_dict,
                       user_answer_per_sum_dict,
                       parts_user_answer_per_sum_dict,
                       content_correct_user_mean_dict,
                       content_correct_count_dict,
                       parts_content_correct_user_mean_dict,
                       last_correct_timestamp_dict,
                       last_incorrect_timestamp_dict,
                       like_answer_dict,
                       like_answer_three_dict,
                       dislike_answer_dict,
                       dislike_answer_three_dict,
                       parts_answered_correctly_cumsum_u_dict,
                       parts_answered_incorrectly_cumsum_u_dict)
print('valid add_user_feats =',(datetime.datetime.now() - start))

In [ ]:
%%cython
import cython
cimport cython
import numpy as np
cimport numpy as np
import pandas as pd

DTYPE = np.int32
ctypedef np.int32_t np_int_t

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
def add_user_content_feats(df, user_content_dict, user_tags_dict, user_repeat_count_dict):
    cdef int arr_size = len(df)
    cdef int cnt,i
    cdef np.ndarray[np_int_t, ndim=1] row
    cdef np.ndarray[np_int_t, ndim=1] uc = np.zeros(arr_size,dtype=DTYPE)
    cdef np.ndarray[np_int_t, ndim=1] rc = np.zeros(arr_size,dtype=DTYPE)
    for cnt,row in enumerate(df[['user_id','content_id','answered_correctly','enc_tags','part']].values):
        if (row[1] in user_content_dict[row[0]]):
            uc[cnt] = user_content_dict[row[0]][row[1]]
        else:
            uc[cnt] = 0
        if (row[3] in user_tags_dict[row[0]]):
            uc[cnt] = uc[cnt] + (user_tags_dict[row[0]][row[3]] * 2)

        rc[cnt] = user_repeat_count_dict[row[4]][row[0]]
        
        if row[2] == 0:
            user_content_dict[row[0]][row[1]] = 1
            user_tags_dict[row[0]][row[3]] = 1
        else:
            user_content_dict[row[0]][row[1]] = 2
            user_tags_dict[row[0]][row[3]] = 2
            
        if row[1] in user_content_dict[row[0]]:
            if not row[0] in user_repeat_count_dict[row[4]]:
                user_repeat_count_dict[row[4]][row[0]] = 0
            user_repeat_count_dict[row[4]][row[0]] = user_repeat_count_dict[row[4]][row[0]] + 1
    df['done_content_tag'] = uc
    df['done_content_tag'] = df['done_content_tag'].astype('uint8')
    df['repeat_part_per'] = rc
    return df


In [ ]:
#train add_user_content_feats = 0:01:37.861307
#valid add_user_content_feats = 0:00:08.414493
#回答数
user_content_dict = defaultdict(dict)
user_tags_dict = defaultdict(dict)
user_repeat_count_dict = {}
for p in range(0,7):
    user_repeat_count_dict[p] = defaultdict(int)
start = datetime.datetime.now()
train = add_user_content_feats(train, user_content_dict, user_tags_dict, user_repeat_count_dict)
print('train add_user_content_feats =',(datetime.datetime.now() - start))
start = datetime.datetime.now()
valid = add_user_content_feats(valid, user_content_dict, user_tags_dict, user_repeat_count_dict)
print('valid add_user_content_feats =',(datetime.datetime.now() - start))

In [ ]:
%%cython
import cython
cimport cython
import numpy as np
cimport numpy as np
import pandas as pd

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
def post_round(df,list use_tags,list use_content):
    df.loc[~df['enc_tags'].isin(use_tags),'enc_tags'] = 65535
    df.loc[~df['content_id'].isin(use_content),'content_id'] = 532 #暫定
    return df

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
def post_features(df):
    cdef int i
    df['lag_mean'] = (df['lag_sum'] / df['count_u'])
    df.loc[df['lag_mean'] > 65535,'lag_mean'] = 65535
    df['lag_mean'] = df['lag_mean'].astype('float16')
    
    df['elapsed_time_per_mean'] = df['elapsed_time_per_mean'] / df['count_u']
    df['elapsed_time_per_mean'].fillna(1,inplace=True)
    df['elapsed_time_per_mean'] = df['elapsed_time_per_mean'].astype('float16')
    
    cdef list parts = list(df['part'].unique())
    cdef str p
    for i in parts:
        p = str(i)
        df.loc[df['part']==i,'part_elapsed_time_per_mean'] = df['part_elapsed_time_per_mean'] / df['p' + p + '_count_u']
        df.loc[df['p' + p  + '_count_u']>0,'repeat_part_per'] = df['repeat_part_per'] / df['p' + p  + '_count_u']

    df['prior_question_elapsed_time'] = df['prior_question_elapsed_time'] / df['part_elapsed_time_per_mean']
    
    df.loc[df['part_user_answer_per_mean'] == 0, 'part_user_answer_per_mean'] = np.nan
    df['repeat_part_per'] = df['repeat_part_per'].astype('float16')
    return df

In [ ]:
#少数カテゴリ値を丸める。推論時に未知のカテゴリが出てきた場合にも丸めます
round_max = np.iinfo(np.uint16).max

use_tags = list(train['enc_tags'].value_counts()[train['enc_tags'].value_counts()>3].index)
use_content = list(train['content_id'].value_counts()[train['content_id'].value_counts()>3].index)

train = post_round(train,use_tags,use_content)
valid = post_round(valid,use_tags,use_content)

train = post_features(train)
valid = post_features(valid)

In [ ]:
print(train.info())
print(train.memory_usage(deep=True))
'''
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26742859 entries, 0 to 26742858
Data columns (total 67 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          uint32 
 1   timestamp                       uint32 
 2   user_id                         int32  
 3   content_id                      uint16 
 4   content_type_id                 uint8  
 5   answered_correctly              uint8  
 6   prior_question_elapsed_time     float32
 7   prior_question_had_explanation  uint8  
 8   user_answer                     uint8  
 9   correct_answer                  uint8  
 10  part                            uint8  
 11  tags_92                         uint8  
 12  enc_tags                        uint16 
 13  answered_correctly_avg_c        uint8  
 14  lag_time                        uint16 
 15  lag_day                         uint16 
 16  lecture_count                   uint8  
 17  prior_content_id                uint16 
 18  prior_content_diff              uint16 
 19  prior_root                      float16
 20  is_same_prior                   uint8  
 21  lag_time_per                    float32
 22  elapsed_lag_per                 float16
 23  elapsed_time_per_mean           float16
 24  part_elapsed_time_per_mean      float16
 25  prior_prior_lag_time            uint16 
 26  prior_lag_time                  uint16 
 27  lag_lag_time                    float16
 28  lag_sum                         int32  
 29  prior_avg_c                     uint8  
 30  prior_prior_avg_c               uint8  
 31  answered_correctly_sum_u        uint16 
 32  answered_cumsum_u               int16  
 33  part_answered_cumsum_u          int16  
 34  count_u                         uint16 
 35  answered_correctly_avg_u        float16
 36  answered_diff_mean              float16
 37  avg_c_mean                      float16
 38  part_avg_c_mean                 uint8  
 39  avg_c_per_u                     float16
 40  user_answer_per_mean            float16
 41  part_user_answer_per_mean       float16
 42  content_lv                      float16
 43  part_content_lv                 float16
 44  last_correct_timelag            uint32 
 45  last_incorrect_timelag          uint32 
 46  is_like_answer                  float16
 47  is_dislike_answer               float16
 48  part_count_per                  float16
 49  lr_count_per                    float16
 50  p0_count_u                      uint32 
 51  p0_mean_u                       float16
 52  p1_count_u                      uint32 
 53  p1_mean_u                       float16
 54  p2_count_u                      uint32 
 55  p2_mean_u                       float16
 56  p3_count_u                      uint32 
 57  p3_mean_u                       float16
 58  p4_count_u                      uint32 
 59  p4_mean_u                       float16
 60  p5_count_u                      uint32 
 61  p5_mean_u                       float16
 62  p6_count_u                      uint32 
 63  p6_mean_u                       float16
 64  done_content_tag                uint8  
 65  repeat_part_per                 float16
 66  lag_mean                        float16
dtypes: float16(26), float32(2), int16(2), int32(2), uint16(10), uint32(11), uint8(14)
memory usage: 3.7 GB
None
Index                     128
row_id              106971436
timestamp           106971436
user_id             106971436
content_id           53485718
                      ...    
p6_count_u          106971436
p6_mean_u            53485718
done_content_tag     26742859
repeat_part_per      53485718
lag_mean             53485718
Length: 68, dtype: int64
'''

In [ ]:
TARGET = 'answered_correctly'
FEATS = ['answered_correctly_avg_u','prior_root',#, 'answered_correctly_sum_u',, 'prior_question_had_explanation' 
         'answered_cumsum_u',#'q_elapsed_time_mean',#'prior_prior_question_elapsed_time_per','q_elapsed_time_per',
         'answered_correctly_avg_c','avg_c_per_u','part_avg_c_mean','elapsed_time_per_mean','part_answered_cumsum_u',
         'part','lag_mean','user_answer_per_mean','part_user_answer_per_mean',#'lag_time_per',
         'count_u','prior_question_elapsed_time','lag_time','prior_avg_c',#'prior_prior_avg_c',#,'lag_day',,'is_same_prior_content'
         'part_count_per','lr_count_per','part_elapsed_time_per_mean','last_incorrect_timelag','last_correct_timelag',
         'p0_mean_u','p1_mean_u','p2_mean_u','p3_mean_u','p4_mean_u','p5_mean_u','p6_mean_u','done_content_tag','avg_c_mean',#'done_tags',
         'content_id','lecture_count','content_lv','part_content_lv','is_same_prior','prior_content_diff',#'enc_tags'
         'answered_diff_mean','prior_lag_time','prior_prior_lag_time','elapsed_lag_per','is_like_answer','is_dislike_answer','repeat_part_per']#,'lag_lag_time','lag_tail_half_mean'
#categorical_feature:high-cardinalityなカテゴリ変数
CATEGORICAL = ['part','content_id']
dro_cols = list(set(train.columns) - set(FEATS))
y_tr = train[TARGET]
y_va = valid[TARGET]
train.drop(dro_cols, axis=1, inplace=True)
valid.drop(dro_cols, axis=1, inplace=True)
_=gc.collect()

In [ ]:
num_cols = list(set(FEATS) - set(CATEGORICAL))
for f in FEATS:
    train[f] = train[f].values.astype(np.float32)
    valid[f] = valid[f].values.astype(np.float32)
_=gc.collect()

In [ ]:
#MemorySpike前のclean up
del answered_correctly_sum_u_dict,answered_correctly_cumsum_u_dict,answered_incorrectly_cumsum_u_dict
del count_u_dict,parts_count_u_dict,parts_u_dict,user_content_dict, user_tags_dict
del prior_lag_dict, prior_prior_lag_dict, avg_c_sum_u_dict,user_answer_per_sum_dict, parts_user_answer_per_sum_dict
del lag_sum_dict,prior_time_dict,prior_time_per_sum_dict,parts_avg_c_dict,lect_u_dict
del content_correct_user_mean_dict,content_correct_count_dict,parts_content_correct_user_mean_dict
del last_correct_timestamp_dict,last_incorrect_timestamp_dict,like_answer_dict,like_answer_three_dict, dislike_answer_dict, dislike_answer_three_dict
del parts_answered_correctly_cumsum_u_dict,parts_answered_incorrectly_cumsum_u_dict,user_repeat_count_dict

valid.to_pickle('valid.pickle')
del valid
_=gc.collect()

In [ ]:
# [Tips] https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/55325
lgb_train = lgb.Dataset(train[FEATS], label=y_tr, feature_name=FEATS, categorical_feature=CATEGORICAL)
print('Dataset Constructed')
lgb_train.raw_data=None
del train, y_tr
_=gc.collect()
lgb_train.save_binary('train_data.bin')
lgb_train = lgb.Dataset('train_data.bin')

In [ ]:
valid = pd.read_pickle('valid.pickle')
lgb_valid  = lgb.Dataset(valid[FEATS], label=y_va, feature_name=FEATS, categorical_feature=CATEGORICAL, reference=lgb_train)
lgb_valid.raw_data=None
_=gc.collect()
lgb_valid.save_binary('valid_data.bin')
lgb_valid = lgb.Dataset('valid_data.bin')

In [ ]:
'''
Training until validation scores don't improve for 10 rounds
[10]	training's auc: 0.771457	valid_1's auc: 0.773254
[20]	training's auc: 0.776792	valid_1's auc: 0.778237
[30]	training's auc: 0.77924	valid_1's auc: 0.780527
[40]	training's auc: 0.781663	valid_1's auc: 0.782359
[50]	training's auc: 0.783615	valid_1's auc: 0.783822
[60]	training's auc: 0.785135	valid_1's auc: 0.784998
[70]	training's auc: 0.786532	valid_1's auc: 0.785971
[80]	training's auc: 0.787862	valid_1's auc: 0.786724
[90]	training's auc: 0.788684	valid_1's auc: 0.787326
[100]	training's auc: 0.789814	valid_1's auc: 0.787998
[110]	training's auc: 0.790783	valid_1's auc: 0.78862
[120]	training's auc: 0.791562	valid_1's auc: 0.789039
[130]	training's auc: 0.792353	valid_1's auc: 0.789446
[140]	training's auc: 0.793102	valid_1's auc: 0.789791
[150]	training's auc: 0.793807	valid_1's auc: 0.790098
[160]	training's auc: 0.794459	valid_1's auc: 0.79032
[170]	training's auc: 0.794939	valid_1's auc: 0.790598
[180]	training's auc: 0.795422	valid_1's auc: 0.790802
[190]	training's auc: 0.79589	valid_1's auc: 0.790938
[200]	training's auc: 0.796324	valid_1's auc: 0.791165
[210]	training's auc: 0.79669	valid_1's auc: 0.791364
[220]	training's auc: 0.797034	valid_1's auc: 0.791546
[230]	training's auc: 0.797332	valid_1's auc: 0.791711
[240]	training's auc: 0.797628	valid_1's auc: 0.791806
[250]	training's auc: 0.797834	valid_1's auc: 0.791886
[260]	training's auc: 0.798119	valid_1's auc: 0.79205
[270]	training's auc: 0.79832	valid_1's auc: 0.792104
[280]	training's auc: 0.798571	valid_1's auc: 0.792239
[290]	training's auc: 0.798764	valid_1's auc: 0.792327
[300]	training's auc: 0.798994	valid_1's auc: 0.792462
[310]	training's auc: 0.799199	valid_1's auc: 0.792573
[320]	training's auc: 0.799337	valid_1's auc: 0.792614
[330]	training's auc: 0.799499	valid_1's auc: 0.79267
[340]	training's auc: 0.799664	valid_1's auc: 0.792735
[350]	training's auc: 0.799802	valid_1's auc: 0.792778
[360]	training's auc: 0.799954	valid_1's auc: 0.792852
[370]	training's auc: 0.800113	valid_1's auc: 0.792922
[380]	training's auc: 0.800264	valid_1's auc: 0.792988
[390]	training's auc: 0.800418	valid_1's auc: 0.793066
[400]	training's auc: 0.800556	valid_1's auc: 0.793128
[410]	training's auc: 0.800692	valid_1's auc: 0.793165
[420]	training's auc: 0.800842	valid_1's auc: 0.793236
[430]	training's auc: 0.800973	valid_1's auc: 0.793294
[440]	training's auc: 0.801097	valid_1's auc: 0.793344
[450]	training's auc: 0.801238	valid_1's auc: 0.793414
[460]	training's auc: 0.801364	valid_1's auc: 0.793441
[470]	training's auc: 0.801504	valid_1's auc: 0.793494
[480]	training's auc: 0.80163	valid_1's auc: 0.793549
[490]	training's auc: 0.801747	valid_1's auc: 0.793594
[500]	training's auc: 0.801871	valid_1's auc: 0.793645
[510]	training's auc: 0.801976	valid_1's auc: 0.793667
[520]	training's auc: 0.802099	valid_1's auc: 0.793706
[530]	training's auc: 0.802214	valid_1's auc: 0.79373
[540]	training's auc: 0.802321	valid_1's auc: 0.793758
[550]	training's auc: 0.802421	valid_1's auc: 0.793775
[560]	training's auc: 0.802532	valid_1's auc: 0.793807
[570]	training's auc: 0.80264	valid_1's auc: 0.793844
[580]	training's auc: 0.80275	valid_1's auc: 0.793865
[590]	training's auc: 0.802856	valid_1's auc: 0.793901
[600]	training's auc: 0.802957	valid_1's auc: 0.793925
[610]	training's auc: 0.803067	valid_1's auc: 0.793965
[620]	training's auc: 0.803182	valid_1's auc: 0.794009
[630]	training's auc: 0.803288	valid_1's auc: 0.794046
[640]	training's auc: 0.803392	valid_1's auc: 0.794071
[650]	training's auc: 0.803492	valid_1's auc: 0.794098
[660]	training's auc: 0.803582	valid_1's auc: 0.794113
[670]	training's auc: 0.803689	valid_1's auc: 0.794155
[680]	training's auc: 0.803782	valid_1's auc: 0.794163
[690]	training's auc: 0.803875	valid_1's auc: 0.794178
[700]	training's auc: 0.803977	valid_1's auc: 0.794207
[710]	training's auc: 0.804071	valid_1's auc: 0.794229
[720]	training's auc: 0.804159	valid_1's auc: 0.79425
[730]	training's auc: 0.804247	valid_1's auc: 0.794266
[740]	training's auc: 0.804344	valid_1's auc: 0.794292
[750]	training's auc: 0.804428	valid_1's auc: 0.794302
[760]	training's auc: 0.804517	valid_1's auc: 0.794317
[770]	training's auc: 0.804603	valid_1's auc: 0.794337
[780]	training's auc: 0.80469	valid_1's auc: 0.794349
[790]	training's auc: 0.804774	valid_1's auc: 0.794358
[800]	training's auc: 0.804854	valid_1's auc: 0.794363
[810]	training's auc: 0.804941	valid_1's auc: 0.794375
[820]	training's auc: 0.805033	valid_1's auc: 0.794391
[830]	training's auc: 0.805117	valid_1's auc: 0.794397
[840]	training's auc: 0.805202	valid_1's auc: 0.794419
[850]	training's auc: 0.805284	valid_1's auc: 0.794427
[860]	training's auc: 0.805372	valid_1's auc: 0.794442
[870]	training's auc: 0.805445	valid_1's auc: 0.794449
[880]	training's auc: 0.805524	valid_1's auc: 0.794461
[890]	training's auc: 0.805599	valid_1's auc: 0.794467
[900]	training's auc: 0.805678	valid_1's auc: 0.794474
[910]	training's auc: 0.805767	valid_1's auc: 0.794486
[920]	training's auc: 0.805854	valid_1's auc: 0.794504
[930]	training's auc: 0.80593	valid_1's auc: 0.79451
[940]	training's auc: 0.806008	valid_1's auc: 0.794517
[950]	training's auc: 0.806091	valid_1's auc: 0.794528
[960]	training's auc: 0.80617	valid_1's auc: 0.794535
[970]	training's auc: 0.806254	valid_1's auc: 0.794554
[980]	training's auc: 0.806342	valid_1's auc: 0.794568
[990]	training's auc: 0.806415	valid_1's auc: 0.794575
[1000]	training's auc: 0.806486	valid_1's auc: 0.794576
[1010]	training's auc: 0.80656	valid_1's auc: 0.794588
[1020]	training's auc: 0.806635	valid_1's auc: 0.794594
[1030]	training's auc: 0.806703	valid_1's auc: 0.794602
[1040]	training's auc: 0.806768	valid_1's auc: 0.794603
[1050]	training's auc: 0.806855	valid_1's auc: 0.794613
[1060]	training's auc: 0.80694	valid_1's auc: 0.794624
[1070]	training's auc: 0.807018	valid_1's auc: 0.794627
[1080]	training's auc: 0.807093	valid_1's auc: 0.794637
[1090]	training's auc: 0.807166	valid_1's auc: 0.794638
[1100]	training's auc: 0.807245	valid_1's auc: 0.794647
[1110]	training's auc: 0.807322	valid_1's auc: 0.794659
[1120]	training's auc: 0.807397	valid_1's auc: 0.794664
[1130]	training's auc: 0.807483	valid_1's auc: 0.794665
[1140]	training's auc: 0.807551	valid_1's auc: 0.794666
[1150]	training's auc: 0.807628	valid_1's auc: 0.794675
[1160]	training's auc: 0.807716	valid_1's auc: 0.794685
[1170]	training's auc: 0.807791	valid_1's auc: 0.794695
[1180]	training's auc: 0.807868	valid_1's auc: 0.794709
[1190]	training's auc: 0.807942	valid_1's auc: 0.794717
[1200]	training's auc: 0.808015	valid_1's auc: 0.794724
[1210]	training's auc: 0.808079	valid_1's auc: 0.794726
[1220]	training's auc: 0.808146	valid_1's auc: 0.794734
[1230]	training's auc: 0.808213	valid_1's auc: 0.794737
[1240]	training's auc: 0.808284	valid_1's auc: 0.79474
[1250]	training's auc: 0.808348	valid_1's auc: 0.794745
[1260]	training's auc: 0.808425	valid_1's auc: 0.794763
[1270]	training's auc: 0.808496	valid_1's auc: 0.794771
[1280]	training's auc: 0.808561	valid_1's auc: 0.794781
Early stopping, best iteration is:
[1279]	training's auc: 0.808556	valid_1's auc: 0.794781
'''

start = datetime.datetime.now()

params = {
'num_leaves': 350,
'max_bin':700,
'min_child_weight': 0.03454472573214212,
'feature_fraction': 0.58,
'bagging_fraction': 0.58,
'objective': 'binary',
'max_depth': 16,
'learning_rate': 0.05,
"boosting_type": "gbdt",
"bagging_seed": 11,
"metric": 'auc',
"verbosity": -1,
'reg_alpha': 0.3899927210061127,
'reg_lambda': 0.6485237330340494,
'random_state': 47,
}

if debug:
    params['learning_rate'] = 0.07

file = 'lgb.pkl'
if build:
    model = lgb.train(params,
                    lgb_train,
                    valid_sets=[lgb_train, lgb_valid],
                    verbose_eval=10,
                    num_boost_round=2000,
                    early_stopping_rounds=10
                )
    display(pd.DataFrame(model.feature_importance(), index=FEATS, columns=['importance']).sort_values('importance',ascending=False))
    pickle.dump(model, open(file, 'wb'))
    del model
    _=gc.collect()
else:
    file = root + 'lgb.pkl'
    model = pickle.load(open(file, 'rb')) 

print('train phase =',(datetime.datetime.now() - start))